<a href="https://colab.research.google.com/github/Go-pulkit/GEE_practice/blob/main/Copy_of_GEE__basics_NDVI_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my second hands on experience with GEE Python API.
In this, I shall be importing a DEM dataset and Global Forest cover dataset to make a primary visual assessment of the Andean Region. 
In the coming days, i shall extend this to more sophisticated analysis.

Step 1: Let's import and initialize the EE Python API

In [3]:
import ee

ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=eorP3F5GMMxWowwiRDXGQ0wAPM5wJDJlKZTc83Xna8s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWisV_g-VEhgMBV1s72yCr1AhZqnBXSOs-8rSQF6ND86FpAuTq-2QJ0

Successfully saved authorization token.


In [32]:
#After succesful authentication, we can initialize the API to start working.
ee.Initialize()

Get the coordinates of our area of interest


In [33]:
geojson ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -83.671875,
              -54.05938788662357
            ],
            [
              -35.33203125,
              -54.87660665410867
            ],
            [
              -33.39843750000001,
              -8.233237111274553
            ],
            [
              -36.2109375,
              12.039320557540572
            ],
            [
              -83.671875,
              12.382928338487408
            ],
            [
              -83.671875,
              -54.05938788662357
            ]
          ]
        ]
      }
    }
  ]
}

In [34]:
coords = geojson['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

Import SRTM dataset and clip it to AOI 

In [35]:
dem = ee.Image('USGS/SRTMGL1_003').clip(aoi)

Mask our DEM to greater than 1800 meters and set it up for visualization

In [36]:
# Make pixels with elevation below sea level transparent.
elv_img = dem.updateMask(dem.gt(1800))
print(elv_img.getInfo())
# Display the thumbnail of styled elevation in France.
elevis = {
    'min': 1800, 'max': 8000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

rgb = elv_img.visualize(**elevis) 
url = rgb.getThumbURL({'dimensions': 512})
print(url)    

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [180986, 249508], 'origin': [346781, 168093], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'id': 'USGS/SRTMGL1_003', 'version': 1641990767055141, 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-83.671875, -54.05938788662357], [-35.33203125, -54.87660665410867], [-33.39843750000001, -8.233237111274553], [-36.2109375, 12.039320557540572], [-83.671875, 12.382928338487408], [-83.671875, -54.05938788662357]]]}, 'system:visualization_0_min': '0.0', 'type_name': 'Image', 'keywords': ['dem', 'elevation', 'geophysical', 'nasa', 'srtm', 'topography', 'usgs'], 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10

In [37]:
import folium

# Define the center of our map.
lat, lon = -32.338200, -70.051575


Add a foilum visualization method

In [38]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

Create a folium map instance for elevation and hillshade visualization

In [39]:
my_map = folium.Map(location=[lat, lon], zoom_start=7)

create hillshade

In [40]:
exaggeration = 5
hillshade = ee.Terrain.hillshade(elv_img.multiply(exaggeration))
hillvis = {'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

Visualize

In [41]:


my_map.add_ee_layer(elv_img, elevis, 'Elevation')
my_map.add_ee_layer(hillshade, hillvis, 'Hillshade')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [42]:
my_map.save('SA_SRTM_interactive.html')

Create a study area mask from the elevation raster



In [43]:
#elv_stuff = elv_img.reduce(ee.Reducer.mean())
elv_mask = elv_img.updateMask(mask=0.5)
elvector = elv_mask.toInt().reduceToVectors(reducer = ee.Reducer.mean(),
        geometry= aoi, 
        scale= 30,
        maxPixels= 1e16)
viz = {'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}


In [44]:

my_map_3 = folium.Map(location=[lat, lon], zoom_start=4)
my_map_3.add_ee_layer(elv_mask, viz, 'Polygon')
display(my_map_3)

In [45]:
def maskS2clouds(image): 
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-01-01', '2020-12-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)).map(maskS2clouds).mosaic().clip(aoi).updateMask(elv_mask)
visualization = {'min': 0.0, 'max': 0.3,'bands': ['B5', 'B7', 'B4']}


In [46]:
my_map_2 = folium.Map(location=[lat, lon], zoom_start=7)

**Let's make a NDVI time series from LANDSAT 8 dataset from 2014-2021 and also create False color composite visualisations.**

In [47]:
#Import imagery and create a cloud mask from the pixel_qa band of LS8 data
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(aoi)

In [48]:
def maskL8SR(image_collection):
  cloudShadowBitMask = 1 << 3
  cloudBitMask = 1 << 5
  qa = image_collection.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).bitwiseAnd(qa.bitwiseAnd(cloudBitMask).eq(0))
  return image_collection.updateMask(mask).divide(10000).select('B[0-9]').copyProperties(image_collection, ['system:time_start'])

In [49]:
step_list = ee.List.sequence(2014,2021)
def func(year):
  startDate = ee.Date.fromYMD(year,9,1)
  endDate = ee.Date.fromYMD(year,10,30)
  composite_i = image_collection.filterDate(startDate,endDate).map(maskL8SR).median().set('system:time_start', startDate)
  return composite_i                                

filter_collection = step_list.map(func)                                   

yearly_composite = ee.ImageCollection(filter_collection) 
print(yearly_composite)                                    

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.fromImages",
    "arguments": {
      "images": {
        "functionInvocationValue": {
          "functionName": "List.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_1_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Element.set",
                    "arguments": {
                      "key": {
                        "constantValue": "system:time_start"
                      },
                      "object": {
                        "functionInvocationValue": {
                          "functionName": "reduce.median",
                          "arguments": {
                            "collection": {
                              "functionInvocationValue": {
                              

In [50]:
def ndvi(img):
  ndvi_img = img.select(['B4','B5'], ['red', 'nir'])
  ndvi_img = ndvi_img.expression('(NIR - RED)/(RED + NIR)', {'NIR' : ndvi_img.select(['nir']), 'RED' : ndvi_img.select(['red'])}).rename(['NDVI'])
  return img.addBands(ndvi_img)

In [51]:
def funk(image):
  return ndvi(image)
yearly_composite = yearly_composite.map(funk)

In [52]:
ndvi_collection = yearly_composite.select(['NDVI'])
y2014 = ndvi_collection.filterDate('2014-01-01','2014-12-31').first().clip(aoi).updateMask(elv_mask)
y2015 = ndvi_collection.filterDate('2015-01-01','2015-12-31').first().clip(aoi).updateMask(elv_mask)
y2016 = ndvi_collection.filterDate('2016-01-01','2016-12-31').first().clip(aoi).updateMask(elv_mask)
y2017 = ndvi_collection.filterDate('2017-01-01','2017-12-31').first().clip(aoi).updateMask(elv_mask)
y2018 = ndvi_collection.filterDate('2018-01-01','2018-12-31').first().clip(aoi).updateMask(elv_mask)
y2019 = ndvi_collection.filterDate('2019-01-01','2019-12-31').first().clip(aoi).updateMask(elv_mask)
y2020 = ndvi_collection.filterDate('2020-01-01','2020-12-31').first().clip(aoi).updateMask(elv_mask)
#y2021 = ndvi_collection.filterDate('2021-01-01','2021-12-31').first().clip(aoi).updateMask(elv_mask)

ndvi_params = {'min':0, 'max':1, 'palette':['red','yellow', 'green']}

my_map_4 = folium.Map(location=[lat, lon], zoom_start=3)

In [53]:
my_map_4.add_ee_layer(y2014, ndvi_params, 'NDVI 2014')
my_map_4.add_ee_layer(y2015, ndvi_params, 'NDVI 2015')
my_map_4.add_ee_layer(y2016, ndvi_params, 'NDVI 2016')
my_map_4.add_ee_layer(y2017, ndvi_params, 'NDVI 2017')
my_map_4.add_ee_layer(y2018, ndvi_params, 'NDVI 2018')
my_map_4.add_ee_layer(y2019, ndvi_params, 'NDVI 2019')
my_map_4.add_ee_layer(y2020, ndvi_params, 'NDVI 2020')
#my_map_4.add_ee_layer(y2021, ndvi_params, 'NDVI 2021')

# Add a layer control panel to the map.
my_map_4.add_child(folium.LayerControl())

# Display the map.
display(my_map_4)

In [54]:
y2014 = yearly_composite.filterDate('2014-01-01','2014-12-31').first().clip(aoi).updateMask(elv_mask)
y2015 = yearly_composite.filterDate('2015-01-01','2015-12-31').first().clip(aoi).updateMask(elv_mask)
y2016 = yearly_composite.filterDate('2016-01-01','2016-12-31').first().clip(aoi).updateMask(elv_mask)
y2017 = yearly_composite.filterDate('2017-01-01','2017-12-31').first().clip(aoi).updateMask(elv_mask)
y2018 = yearly_composite.filterDate('2018-01-01','2018-12-31').first().clip(aoi).updateMask(elv_mask)
y2019 = yearly_composite.filterDate('2019-01-01','2019-12-31').first().clip(aoi).updateMask(elv_mask)
y2020 = yearly_composite.filterDate('2020-01-01','2020-12-31').first().clip(aoi).updateMask(elv_mask)
#y2021 = ndvi_collection.filterDate('2021-01-01','2021-12-31').first().clip(aoi).updateMask(elv_mask)


my_map_5 = folium.Map(zoom_start=3)

In [55]:
viz_params = {
    'bands': ['B7', 'B5', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Define a map centered on San Francisco Bay.
map_5 = folium.Map(zoom_start=5)

# Add the image layer to the map and display it.
map_5.add_ee_layer(y2020, viz_params, 'false color composite')
display(map_5)